In [1]:
require 'daru'
require 'daru_plotting_plotly'
require 'hg'

class Numeric
  def mili
    (self * 10 ** (-3)).to_f
  end
  
  def micro
    (self * 10 ** (-6)).to_f
  end
end

Daru.plotting_library = :plotly

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

:plotly

In [2]:
class LeastSquareFitting
  def initialize(xs, ys)
    raise ArgumentError "xs and ys should be same size" if xs.size != ys.size
    @xs = xs
    @ys = ys
  end
  
  def fit
    xy = @xs.zip(@ys).map { |x, y| x * y }.sum
    x = @xs.sum
    y = @ys.sum
    x2 = @xs.sum { |x| x ** 2 }
    n = @xs.size
    
    @a = (n*xy - x*y)/(n*x2 - x**2)
    @b = (x2*y - xy*x)/(n*x2 - x**2)
  end
  
  def sigma_y
    @sigma_y ||= Math.sqrt(1.0/(@xs.size - 2)*(@xs.zip(@ys).map {|x, y| (x * @a + @b - y) ** 2}.sum))
  end
  
  def sigma_a
    x = @xs.sum
    x2 = @xs.sum { |x| x ** 2 }
    @sigma_a ||= sigma_y * Math.sqrt(@xs.size.to_f/(@xs.size.to_f*x2 - x**2))
  end
  
  def sigma_b
    x = @xs.sum
    x2 = @xs.sum { |x| x ** 2 }
    @sigma_b ||= sigma_y * Math.sqrt(x2/(@xs.size.to_f*x2 - x**2))
  end
  
  attr_reader :a, :b
end

class ExpFitting < LeastSquareFitting
  def initialize(xs, ys)
    super(xs, ys.map{ |y| Math.log(y) })
  end
  
  def to_s
    "log y = #{'%1.2e' % @a} x #{ @b.positive? ? '+' : '-' } #{'%1.2e' % @b.abs }"
  end
  
  def points(min, max)
    ys = [min, max].map { |x| Math.exp(@b) * Math.exp( @a * x ) }
    { x: [min, max], y: ys, name: to_s }
  end
end

class LinearFitting < LeastSquareFitting
  def points(min, max)
    ys = [min, max].map { |x| x * @a + @b }
    { x: [min, max], y: ys , name: to_s}
  end
  
  def to_s
    "y = #{'%1.2f' % @a} x + #{'%1.2f' % @b}"
  end
end

:to_s

In [3]:
Daru::DataFrame.from_csv('data/nakagawa/dark_solar_battery.csv').tap do |df|
  line = df.filter_rows { |row| row['v'] < 0.5 }
  f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
  f.fit
  df.map_rows! do |row|
    row['i'] = row['i'] - (f.a * row['v'] + f.b)
    row
  end
  plot = df.plot(x: 'v', y: 'i', mode: [:markers, :lines])
  
  line = df.filter_rows { |row| row['v'] > 0.75 }
  f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
  f.fit
  puts "直列抵抗 = #{(1/f.a)} Ω"
  
  plot.data.push f.points(0, 1)
  plot.show
  df.map_rows! do |row|
    if row['v'] > 0.4
      row['v'] = row['v'] - ((1/f.a) * row['i'])
    end
    row
  end
  df.plot(x: 'v', y: 'i', mode: [:markers, :lines]).show
  
  df.filter_rows{|row| row['v'] >= 0.5}.plot(x: 'v', y: 'i', layout: { yaxis: { type: :log } }).tap do |plot|
    line = df.filter_rows do |row|
      if row['i'] < 0
        false
      else
        0.5 < row['v'] && row['v'] < 0.703 && row['i'] < 0.002
      end
    end
    f = ExpFitting.new(line['v'].to_a, line['i'].to_a)
    f.fit
    plot.data.push f.points(0.54, 0.7)
    plot.show
  end
end
nil

直列抵抗 = 44.598495337653006 Ω


#<Plotly::Offline::HTML:0x007fcdad1def20 @id="2b10e5ed-a352-4491-b32a-343320a97e8a", @data=[{:x=>[0.844, 0.815, 0.785, 0.704, 0.542, 0.455, 0.348, 0.275, 0.207, 0.146, 0.121, 0.0903, 0.0783, 0.06559999999999999, 0.022, 0.0007, -0.0003, -0.0158, -0.028300000000000002, -0.0594, -0.07790000000000001, -0.103, -0.115, -0.143, -0.159, -0.184, -0.203, -0.617, -1.218], :y=>[0.0031695001737205254, 0.0025039150706401784, 0.0018464132398674036, 0.0009011582967809175, 0.00011064841060794313, 1.389310136690105e-05, -2.1196761722656107e-05, -3.111788326973751e-05, 1.8544633645310813e-05, 1.1624244407338778e-05, -1.6293947903305234e-05, -1.8137488060776465e-05, -2.1138220369885507e-05, -1.8480662063692687e-05, -1.00499894534563e-05, -8.876289302125148e-06, -8.793016994550915e-06, -6.502296227150312e-06, -5.461392382472387e-06, -7.162361691373043e-08, 3.4689140732096623e-06, 4.359048993322749e-06, 1.3583166842135981e-06, 4.7689941296291845e-05, 7.022298217479765e-06, 9.104105906835615e-06, 3.268627975074609e-05, 7.916101508647754e-05, -6.279232806141039e-05], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0, 1], :y=>[-0.01576007552453625, 0.006662205595691001], :name=>"y = 0.02 x + -0.02"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fcdac07ac70 @id="fb4a3a62-4430-46b8-b851-5074ec6a677c", @data=[{:x=>[0.7026450612796347, 0.7033291553961749, 0.7026527477303929, 0.6638096959025289, 0.537065247375383, 0.45438038858346275, 0.348, 0.275, 0.207, 0.146, 0.121, 0.0903, 0.0783, 0.06559999999999999, 0.022, 0.0007, -0.0003, -0.0158, -0.028300000000000002, -0.0594, -0.07790000000000001, -0.103, -0.115, -0.143, -0.159, -0.184, -0.203, -0.617, -1.218], :y=>[0.0031695001737205254, 0.0025039150706401784, 0.0018464132398674036, 0.0009011582967809175, 0.00011064841060794313, 1.389310136690105e-05, -2.1196761722656107e-05, -3.111788326973751e-05, 1.8544633645310813e-05, 1.1624244407338778e-05, -1.6293947903305234e-05, -1.8137488060776465e-05, -2.1138220369885507e-05, -1.8480662063692687e-05, -1.00499894534563e-05, -8.876289302125148e-06, -8.793016994550915e-06, -6.502296227150312e-06, -5.461392382472387e-06, -7.162361691373043e-08, 3.4689140732096623e-06, 4.359048993322749e-06, 1.3583166842135981e-06, 4.7689941296291845e-05, 7.022298217479765e-06, 9.104105906835615e-06, 3.268627975074609e-05, 7.916101508647754e-05, -6.279232806141039e-05], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fcdad091280 @id="12642879-62e3-41bd-91ec-d62ef786f3a4", @data=[{:x=>[0.7026450612796347, 0.7033291553961749, 0.7026527477303929, 0.6638096959025289, 0.537065247375383], :y=>[0.0031695001737205254, 0.0025039150706401784, 0.0018464132398674036, 0.0009011582967809175, 0.00011064841060794313], :type=>:scatter, :mode=>"", :name=>"i"}, {:x=>[0.54, 0.7], :y=>[0.00011532530549965775, 0.0017191485229164243], :name=>"log y = 1.69e+01 x - 1.82e+01"}], @layout={:width=>1000, :height=>500, :yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [4]:
Daru::DataFrame.from_csv('data/itano/dark_solar_battery.csv').tap do |df|
  line = df.filter_rows { |row| -4 < row['v'] && row['v'] < 0.3 }
  f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
  f.fit
  df.plot(x: 'v', y: 'i', mode: [:markers, :lines]).tap do |plot|
    plot.data.push f.points(-10, 1)
    plot.show
  end

  df.map_rows! do |row|
    row['i'] = row['i'] - (f.a * row['v'] + f.b)
    row
  end
  plot = df.plot(x: 'v', y: 'i', mode: [:markers, :lines])
  
  line = df.filter_rows { |row| row['v'] > 0.6 }
  f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
  f.fit
  puts "直列抵抗 = #{(1/f.a)} Ω"
  
  plot.data.push f.points(0.4, 0.7)
  plot.show
  df.map_rows! do |row|
    row['v'] = row['v'] - ((1/f.a) * row['i'])
    row
  end
  df.plot(x: 'v', y: 'i', mode: [:markers, :lines]).show
  
  df.filter_rows{|row| row['i'] > 0 && row['v'] > 0 }.plot(x: 'v', y: 'i', layout: { yaxis: { type: :log } }).tap do |plot|
    line = df.filter_rows do |row|
      0.5 < row['v'] && row['v'] < 0.535 && row['i'] > 0
    end
    f = ExpFitting.new(line['v'].to_a, line['i'].to_a)
    f.fit
    plot.data.push f.points(0.4, 0.55)
    plot.show
  end
end
nil

#<Plotly::Offline::HTML:0x007fcdabee0ec8 @id="4aec3eff-0a8d-47c3-9bf7-72e4f7354b12", @data=[{:x=>[-10.2, -3.48, -1.712, -1.506, -1.418, -1.186, -0.95, -0.537, -0.307, -0.1555, -0.1237, -0.0857, -0.0523, -0.0412, -0.0334, -0.0162, -0.0097, -0.0003, 0.142, 0.247, 0.383, 0.466, 0.554, 0.57, 0.588, 0.62, 0.626, 0.636], :y=>[-0.082, -0.0274, -0.0135, -0.011800000000000001, -0.0111, -0.0092, -0.0074, -0.004200000000000001, -0.0024, -0.0012, -0.00096, -0.00067, -0.000407, -0.000321, -0.000261, -0.000127, -7.7e-05, -4.4e-06, 0.0011, 0.0019, 0.0031000000000000003, 0.004, 0.0063, 0.0070999999999999995, 0.008, 0.009800000000000001, 0.010199999999999999, 0.0109], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[-10, 1], :y=>[-0.07858682939141651, 0.007870146935284933], :name=>"y = 0.01 x + 0.00"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

直列抵抗 = 16.388653401720266 Ω


#<Plotly::Offline::HTML:0x007fcdabebbd58 @id="c8f46ecf-ef98-4c58-9e99-97bd3988493f", @data=[{:x=>[-10.2, -3.48, -1.712, -1.506, -1.418, -1.186, -0.95, -0.537, -0.307, -0.1555, -0.1237, -0.0857, -0.0523, -0.0412, -0.0334, -0.0162, -0.0097, -0.0003, 0.142, 0.247, 0.383, 0.466, 0.554, 0.57, 0.588, 0.62, 0.626, 0.636], :y=>[-0.0018412255844616615, -5.8578394955616925e-05, -5.457240819272194e-05, 2.6324216961776115e-05, 3.466840634816504e-05, 0.00011121217836682594, 5.6317049903050614e-05, 1.0250575091442192e-05, 2.513797351321248e-06, 1.1765441579024167e-05, 1.8261827436508518e-06, -6.843371839499704e-06, -6.3581908678477135e-06, -7.601139706610034e-06, -8.90699564736196e-06, -1.0094267721840575e-05, -1.1182481005800527e-05, -1.2463897139527209e-05, -2.6502781802219396e-05, -5.177391946618783e-05, 7.930346413095824e-05, 0.00032694627912039243, 0.0019352904685067812, 0.0026095348665770334, 0.0033680598144060683, 0.004916548610546574, 0.005269390259822916, 0.005890793008616825], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0.4, 0.7], :y=>[-0.008512475128247411, 0.009792872641723113], :name=>"y = 0.06 x + -0.03"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fcdabbac310 @id="a8bd1591-4240-48d9-8625-ecc298e90aa8", @data=[{:x=>[-10.169824792061878, -3.479039978988243, -1.7111056317168323, -1.5064314184678582, -1.41856816849563, -1.1878226178453042, -0.9509229606114684, -0.537167993122342, -0.30704119775351296, -0.1556928197441568, -0.1237299286760339, -0.08558784635082335, -0.05219579781360486, -0.04107542755589032, -0.03325402633548475, -0.01603456854496258, -0.009516734194624616, -9.573350974559589e-05, 0.14243434490513798, 0.24784850482137993, 0.381700323012802, 0.4606417907505138, 0.5222831952799896, 0.5272332375319646, 0.5328020350654367, 0.5394243888890429, 0.5396417893933615, 0.5394578351205019], :y=>[-0.0018412255844616615, -5.8578394955616925e-05, -5.457240819272194e-05, 2.6324216961776115e-05, 3.466840634816504e-05, 0.00011121217836682594, 5.6317049903050614e-05, 1.0250575091442192e-05, 2.513797351321248e-06, 1.1765441579024167e-05, 1.8261827436508518e-06, -6.843371839499704e-06, -6.3581908678477135e-06, -7.601139706610034e-06, -8.90699564736196e-06, -1.0094267721840575e-05, -1.1182481005800527e-05, -1.2463897139527209e-05, -2.6502781802219396e-05, -5.177391946618783e-05, 7.930346413095824e-05, 0.00032694627912039243, 0.0019352904685067812, 0.0026095348665770334, 0.0033680598144060683, 0.004916548610546574, 0.005269390259822916, 0.005890793008616825], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fcdaba47588 @id="b9c4396d-9baa-41f6-96cb-554f0710d299", @data=[{:x=>[0.381700323012802, 0.4606417907505138, 0.5222831952799896, 0.5272332375319646, 0.5328020350654367, 0.5394243888890429, 0.5396417893933615, 0.5394578351205019], :y=>[7.930346413095824e-05, 0.00032694627912039243, 0.0019352904685067812, 0.0026095348665770334, 0.0033680598144060683, 0.004916548610546574, 0.005269390259822916, 0.005890793008616825], :type=>:scatter, :mode=>"", :name=>"i"}, {:x=>[0.4, 0.55], :y=>[3.1822523746081123e-06, 0.008412812474743686], :name=>"log y = 5.25e+01 x - 3.37e+01"}], @layout={:width=>1000, :height=>500, :yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [5]:
Daru::DataFrame.from_csv('data/sangenya/dark_solar_battery.csv').tap do |df|
  line = df.filter_rows { |row| row['v'] > 0.52 }
  f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
  f.fit
  df.plot(x: 'v', y: 'i', mode: [:markers, :lines]).tap do |plot|
    plot.data.push f.points(0.4, 0.7)
    plot.show
  end
  puts "直列抵抗 = #{(1/f.a)} Ω"
  
  df.map_rows! do |row|
    row['v'] = row['v'] - ((1/f.a) * row['i'])
    row
  end
  df.plot(x: 'v', y: 'i', mode: [:markers, :lines]).show
  
  df.filter_rows{ |row| row['v'] > 0.0 }.plot(x: 'v', y: 'i', layout: { yaxis: { type: :log } }, mode: [:markers, :lines]).tap do |plot|
    line = df.filter_rows do |row|
      if row['i'] < 0
        false
      else
        0.17 < row['v'] && row['v'] < 0.44
      end
    end
    f = ExpFitting.new(line['v'].to_a, line['i'].to_a)
    f.fit
    plot.data.push f.points(0.1, 0.45)
    plot.show
  end
end
nil

#<Plotly::Offline::HTML:0x007fcdae2703e8 @id="6e821e90-294a-4dd5-ab86-6f593f81312c", @data=[{:x=>[-0.991, -0.911, -0.874, -0.802, -0.774, -0.734, -0.709, -0.614, -0.538, -0.458, -0.35, -0.142, -0.07529999999999999, -0.0346, -0.0008, 0.105, 0.123, 0.165, 0.19, 0.267, 0.311, 0.339, 0.39, 0.418, 0.44, 0.459, 0.486, 0.505, 0.519, 0.538, 0.554, 0.565, 0.582], :y=>[-0.009000000000000001, -0.0068, -0.005900000000000001, -0.0044, -0.0039, -0.0033, -0.0029, -0.0017, -0.00115, -0.000743, -0.000439, -0.00014, -7.3e-05, -3.3e-05, -8.0e-07, 8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0.4, 0.7], :y=>[-0.002669356014577802, 0.01722614823814969], :name=>"y = 0.07 x + -0.03"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

直列抵抗 = 15.078783437161324 Ω


#<Plotly::Offline::HTML:0x007fcdae1c1eb0 @id="dd616c14-9a1e-422a-a320-29fb40ef8183", @data=[{:x=>[-0.855290949065548, -0.808464272627303, -0.7850351777207482, -0.7356533528764903, -0.7151927445950709, -0.6842400146573676, -0.6652715280322321, -0.5883660681568258, -0.5206593990472645, -0.44679646390618916, -0.34338041407108616, -0.1398889703187974, -0.07419924880908721, -0.034102400146573675, -0.000787936973250271, 0.10498793697325026, 0.12155243679003251, 0.1615318798094529, 0.18659219494320153, 0.2608176987907639, 0.30210351777207484, 0.32738933675338583, 0.3703975815316903, 0.3908581898131096, 0.405318798094529, 0.41527152803223216, 0.42719274459507084, 0.43262183950162564, 0.43606669109561275, 0.4399879076584514, 0.44090912422129014, 0.43984609747156095, 0.4402594356906835], :y=>[-0.009000000000000001, -0.0068, -0.005900000000000001, -0.0044, -0.0039, -0.0033, -0.0029, -0.0017, -0.00115, -0.000743, -0.000439, -0.00014, -7.3e-05, -3.3e-05, -8.0e-07, 8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fcdae142868 @id="158f1397-b96a-4706-b650-3fd7bf158694", @data=[{:x=>[0.10498793697325026, 0.12155243679003251, 0.1615318798094529, 0.18659219494320153, 0.2608176987907639, 0.30210351777207484, 0.32738933675338583, 0.3703975815316903, 0.3908581898131096, 0.405318798094529, 0.41527152803223216, 0.42719274459507084, 0.43262183950162564, 0.43606669109561275, 0.4399879076584514, 0.44090912422129014, 0.43984609747156095, 0.4402594356906835], :y=>[8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0.1, 0.45], :y=>[4.4666291492181116e-05, 0.005503553139002654], :name=>"log y = 1.38e+01 x - 1.14e+01"}], @layout={:width=>1000, :height=>500, :yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [6]:
Daru::DataFrame.from_csv('data/nagura/dark_solar_battery.csv').tap do |df|
  line = df.filter_rows { |row| row['v'] > 0.6 }
  f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
  f.fit
  df.plot(x: 'v', y: 'i', mode: [:markers, :lines]).tap do |plot|
    plot.data.push f.points(0.4, 0.7)
    plot.show
  end
  puts "直列抵抗 = #{(1/f.a)} Ω"
  
  df.map_rows! do |row|
    row['v'] = row['v'] - ((1/f.a) * row['i'])
    row
  end
  df.plot(x: 'v', y: 'i', mode: [:markers, :lines]).show
  
  df.filter_rows{ |row| row['v'] > 0.0 }.plot(x: 'v', y: 'i', layout: { yaxis: { type: :log } }, mode: [:markers, :lines]).tap do |plot|
    line = df.filter_rows do |row|
      if row['i'] < 0
        false
      else
        0.1 < row['v'] && row['v'] < 0.51
      end
    end
    f = ExpFitting.new(line['v'].to_a, line['i'].to_a)
    f.fit
    plot.data.push f.points(0.1, 0.51)
    plot.show
  end
end
nil

#<Plotly::Offline::HTML:0x007fcdae080d58 @id="af718a0a-a69b-4331-814a-c99552484e58", @data=[{:x=>[-1.708, -1.626, -1.4, -1.296, -1.209, -1.111, -1.018, -0.929, -0.814, -0.744, -0.62, -0.499, -0.396, -0.308, -0.115, -0.0594, -0.0292, -0.013800000000000002, -0.0009000000000000001, 0.13, 0.272, 0.342, 0.408, 0.417, 0.479, 0.506, 0.54, 0.572, 0.59, 0.617, 0.651, 0.693], :y=>[-0.010400000000000001, -0.0079, -0.0037, -0.0025, -0.0018000000000000002, -0.0011, -0.00071, -0.00044, -0.00022, -0.00017, -6.8e-05, -2.8e-05, -1.4e-05, -7.4e-06, -2.0e-06, -1.0e-06, -5.0e-07, -3.0e-07, 0, 6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0.4, 0.7], :y=>[-0.0049764949402013765, 0.008477966881324378], :name=>"y = 0.04 x + -0.02"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

直列抵抗 = 22.297435897438188 Ω


#<Plotly::Offline::HTML:0x007fcdabfb1a28 @id="a40140d4-aa5f-4cf1-9efe-c53bca4b7567", @data=[{:x=>[-1.4761066666666427, -1.4498502564102382, -1.3174994871794785, -1.2402564102564047, -1.1688646153846114, -1.086472820512818, -1.002168820512819, -0.9191891282051272, -0.8090945641025635, -0.7402094358974355, -0.6184837743589742, -0.49837567179487174, -0.3956878358974359, -0.30783499897435895, -0.11495540512820512, -0.05937770256410256, -0.02918885128205128, -0.01379331076923077, -0.0009000000000000001, 0.12985952615384613, 0.270729046153846, 0.3381425435897432, 0.3988580512820503, 0.4036215384615371, 0.4500133333333303, 0.46586461538461127, 0.48425641025640453, 0.4961887179487101, 0.503039999999991, 0.5099723076922967, 0.5127558974358832, 0.5101610256410068], :y=>[-0.010400000000000001, -0.0079, -0.0037, -0.0025, -0.0018000000000000002, -0.0011, -0.00071, -0.00044, -0.00022, -0.00017, -6.8e-05, -2.8e-05, -1.4e-05, -7.4e-06, -2.0e-06, -1.0e-06, -5.0e-07, -3.0e-07, 0, 6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fcdad21eaa8 @id="7dbf229e-4926-4307-8f6a-23ee66893c02", @data=[{:x=>[0.12985952615384613, 0.270729046153846, 0.3381425435897432, 0.3988580512820503, 0.4036215384615371, 0.4500133333333303, 0.46586461538461127, 0.48425641025640453, 0.4961887179487101, 0.503039999999991, 0.5099723076922967, 0.5127558974358832, 0.5101610256410068], :y=>[6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0.1, 0.51], :y=>[3.0654722398985193e-06, 0.003936371122808312], :name=>"log y = 1.75e+01 x - 1.44e+01"}], @layout={:width=>1000, :height=>500, :yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# 抵抗？

In [7]:
data = ['itano', 'nakagawa'].map do |member|
  ['dark', '5cm', '15cm', '25cm'].map do |name|
    file_name = "data/#{member}/#{name}_solar_battery.csv"
    if File.exists?(file_name)
      Daru::DataFrame.from_csv(file_name).sort(['v']).to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "#{member}_#{name}", mode: 'markers+lines')
    end
  end
end.flatten
Plotly::Plot.new(data: data, layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)' } }).show
nil

#<Plotly::Offline::HTML:0x007fcdabd115c0 @id="1d10b9a6-4247-4415-8ba9-8454d63fb1a6", @data=[{:x=>[-10.2, -3.48, -1.712, -1.506, -1.418, -1.186, -0.95, -0.537, -0.307, -0.1555, -0.1237, -0.0857, -0.0523, -0.0412, -0.0334, -0.0162, -0.0097, -0.0003, 0.142, 0.247, 0.383, 0.466, 0.554, 0.57, 0.588, 0.62, 0.626, 0.636], :y=>[-0.082, -0.0274, -0.0135, -0.011800000000000001, -0.0111, -0.0092, -0.0074, -0.004200000000000001, -0.0024, -0.0012, -0.00096, -0.00067, -0.000407, -0.000321, -0.000261, -0.000127, -7.7e-05, -4.4e-06, 0.0011, 0.0019, 0.0031000000000000003, 0.004, 0.0063, 0.0070999999999999995, 0.008, 0.009800000000000001, 0.010199999999999999, 0.0109], :mode=>"markers+lines", :name=>"itano_dark"}, {:x=>[0.34, 0.343, 0.356, 0.357, 0.359, 0.359, 0.362, 0.366, 0.371, 0.375, 0.379, 0.384, 0.387, 0.39, 0.393, 0.422], :y=>[-0.0033799999999999998, -0.0033, -0.00317, -0.0029300000000000003, -0.0024100000000000002, -0.0035600000000000002, -0.0025499999999999997, -0.0022, -0.0019, -0.0016, -0.0013000000000000002, -0.001, -0.0008, -0.0006, -0.0004, 0], :mode=>"markers+lines", :name=>"itano_5cm"}, {:x=>[0.253, 0.302, 0.329, 0.352, 0.372, 0.389, 0.403, 0.409, 0.428], :y=>[-0.0025, -0.002, -0.0017, -0.0014, -0.0011, -0.0008, -0.0005, -0.00035, 0], :mode=>"markers+lines", :name=>"itano_15cm"}, {:x=>[0.173, 0.201, 0.222, 0.246, 0.272, 0.293, 0.316, 0.336, 0.37], :y=>[-0.0017, -0.0015, -0.0013000000000000002, -0.0011, -0.0009000000000000001, -0.0007, -0.0005, -0.0003, 0], :mode=>"markers+lines", :name=>"itano_25cm"}, {:x=>[-1.218, -0.617, -0.203, -0.184, -0.159, -0.143, -0.115, -0.103, -0.07790000000000001, -0.0594, -0.028300000000000002, -0.0158, -0.0003, 0.0007, 0.022, 0.06559999999999999, 0.0783, 0.0903, 0.121, 0.146, 0.207, 0.275, 0.348, 0.455, 0.542, 0.704, 0.785, 0.815, 0.844], :y=>[-0.0099, -0.004900000000000001, -0.0016, -0.00147, -0.00127, -0.0011, -0.00092, -0.00082, -0.000618, -0.000472, -0.000226, -0.000126, -3.0e-06, 5.0e-06, 0.000176, 0.0005200000000000001, 0.00062, 0.0007199999999999999, 0.0009699999999999999, 0.0012, 0.0017, 0.0022, 0.0028, 0.0037, 0.0045000000000000005, 0.0066, 0.008199999999999999, 0.0091, 0.01], :mode=>"markers+lines", :name=>"nakagawa_dark"}, {:x=>[0.247, 0.254, 0.258, 0.266, 0.271, 0.277, 0.281, 0.286, 0.292, 0.298, 0.322], :y=>[-0.0024500000000000004, -0.0021000000000000003, -0.0019, -0.0016, -0.0014, -0.0011, -0.00091, -0.0007, -0.0005, -0.00026, 0], :mode=>"markers+lines", :name=>"nakagawa_5cm"}, {:x=>[0.24, 0.254, 0.272, 0.286, 0.299, 0.312, 0.32, 0.329, 0.334, 0.35], :y=>[-0.0024, -0.0021000000000000003, -0.0017, -0.0014, -0.0011, -0.0008, -0.0006, -0.0004, -0.00029, 0], :mode=>"markers+lines", :name=>"nakagawa_15cm"}, {:x=>[0.157, 0.174, 0.195, 0.215, 0.23, 0.256, 0.277, 0.291, 0.314], :y=>[-0.0015600000000000002, -0.0014, -0.0012, -0.001, -0.0008, -0.0006, -0.0004, -0.00025, 0], :mode=>"markers+lines", :name=>"nakagawa_25cm"}], @layout={:yaxis=>{:title=>"電流(I)"}, :xaxis=>{:title=>"電圧(V)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# ダイオード?

In [8]:
data = ['sangenya', 'nagura', 'nakagawa'].map do |member|
  ['dark', '5cm', '15cm', '25cm'].map do |name|
    file_name = "data/#{member}/#{name}_solar_battery.csv"
    if File.exists?(file_name)
      Daru::DataFrame.from_csv(file_name).sort(['v']).to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "#{member}_#{name}", mode: 'markers+lines')
    end
  end
end.flatten
Plotly::Plot.new(data: data, layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)' } }).show
nil

#<Plotly::Offline::HTML:0x007fcdaba472b8 @id="546210e3-2851-4bd9-b59d-37ed0fa111bb", @data=[{:x=>[-0.991, -0.911, -0.874, -0.802, -0.774, -0.734, -0.709, -0.614, -0.538, -0.458, -0.35, -0.142, -0.07529999999999999, -0.0346, -0.0008, 0.105, 0.123, 0.165, 0.19, 0.267, 0.311, 0.339, 0.39, 0.418, 0.44, 0.459, 0.486, 0.505, 0.519, 0.538, 0.554, 0.565, 0.582], :y=>[-0.009000000000000001, -0.0068, -0.005900000000000001, -0.0044, -0.0039, -0.0033, -0.0029, -0.0017, -0.00115, -0.000743, -0.000439, -0.00014, -7.3e-05, -3.3e-05, -8.0e-07, 8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :mode=>"markers+lines", :name=>"sangenya_dark"}, {:x=>[0.309, 0.345, 0.36, 0.37, 0.379, 0.384, 0.388, 0.391, 0.392, 0.416], :y=>[-0.003, -0.0023, -0.0018000000000000002, -0.0014, -0.001, -0.0008, -0.0006, -0.0004, -0.00034, 0], :mode=>"markers+lines", :name=>"sangenya_5cm"}, {:x=>[0.144, 0.212, 0.263, 0.298, 0.321, 0.341, 0.355, 0.374], :y=>[-0.00145, -0.0013000000000000002, -0.0011, -0.0009000000000000001, -0.0007, -0.0005, -0.00031, 0], :mode=>"markers+lines", :name=>"sangenya_15cm"}, {:x=>[0.119, 0.203, 0.27, 0.303, 0.328, 0.339, 0.361], :y=>[-0.0012, -0.001, -0.000804, -0.000616, -0.00041, -0.000295, 0], :mode=>"markers+lines", :name=>"sangenya_25cm"}, {:x=>[-1.708, -1.626, -1.4, -1.296, -1.209, -1.111, -1.018, -0.929, -0.814, -0.744, -0.62, -0.499, -0.396, -0.308, -0.115, -0.0594, -0.0292, -0.013800000000000002, -0.0009000000000000001, 0.13, 0.272, 0.342, 0.408, 0.417, 0.479, 0.506, 0.54, 0.572, 0.59, 0.617, 0.651, 0.693], :y=>[-0.010400000000000001, -0.0079, -0.0037, -0.0025, -0.0018000000000000002, -0.0011, -0.00071, -0.00044, -0.00022, -0.00017, -6.8e-05, -2.8e-05, -1.4e-05, -7.4e-06, -2.0e-06, -1.0e-06, -5.0e-07, -3.0e-07, 0, 6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :mode=>"markers+lines", :name=>"nagura_dark"}, {:x=>[0.18, 0.194, 0.224, 0.247, 0.27, 0.293, 0.31, 0.331, 0.344, 0.35], :y=>[-0.0018000000000000002, -0.0017, -0.0015, -0.0013000000000000002, -0.0011, -0.0009, -0.0007, -0.0005, 0, -0.0003], :mode=>"markers+lines", :name=>"nagura_5cm"}, {:x=>[0.094, 0.154, 0.213, 0.252, 0.278, 0.297, 0.32], :y=>[-0.00094, -0.00086, -0.0007, -0.00055, -0.0004, -0.00026, 0], :mode=>"markers+lines", :name=>"nagura_15cm"}, {:x=>[4.7e-05, 0.121, 0.187, 0.222, 0.249, 0.267, 0.282, 0.339], :y=>[-0.000466, -0.00045, -0.00041, -0.00037, -0.00033, -0.00029, -0.000245, 0], :mode=>"markers+lines", :name=>"nagura_25cm"}, {:x=>[-1.218, -0.617, -0.203, -0.184, -0.159, -0.143, -0.115, -0.103, -0.07790000000000001, -0.0594, -0.028300000000000002, -0.0158, -0.0003, 0.0007, 0.022, 0.06559999999999999, 0.0783, 0.0903, 0.121, 0.146, 0.207, 0.275, 0.348, 0.455, 0.542, 0.704, 0.785, 0.815, 0.844], :y=>[-0.0099, -0.004900000000000001, -0.0016, -0.00147, -0.00127, -0.0011, -0.00092, -0.00082, -0.000618, -0.000472, -0.000226, -0.000126, -3.0e-06, 5.0e-06, 0.000176, 0.0005200000000000001, 0.00062, 0.0007199999999999999, 0.0009699999999999999, 0.0012, 0.0017, 0.0022, 0.0028, 0.0037, 0.0045000000000000005, 0.0066, 0.008199999999999999, 0.0091, 0.01], :mode=>"markers+lines", :name=>"nakagawa_dark"}, {:x=>[0.247, 0.254, 0.258, 0.266, 0.271, 0.277, 0.281, 0.286, 0.292, 0.298, 0.322], :y=>[-0.0024500000000000004, -0.0021000000000000003, -0.0019, -0.0016, -0.0014, -0.0011, -0.00091, -0.0007, -0.0005, -0.00026, 0], :mode=>"markers+lines", :name=>"nakagawa_5cm"}, {:x=>[0.24, 0.254, 0.272, 0.286, 0.299, 0.312, 0.32, 0.329, 0.334, 0.35], :y=>[-0.0024, -0.0021000000000000003, -0.0017, -0.0014, -0.0011, -0.0008, -0.0006, -0.0004, -0.00029, 0], :mode=>"markers+lines", :name=>"nakagawa_15cm"}, {:x=>[0.157, 0.174, 0.195, 0.215, 0.23, 0.256, 0.277, 0.291, 0.314], :y=>[-0.0015600000000000002, -0.0014, -0.0012, -0.001, -0.0008, 

# 効率

In [11]:
data = ['sangenya', 'nagura'].map do |member|
  ['5cm', '15cm', '25cm'].map do |name|
    file_name = "data/#{member}/#{name}_solar_battery.csv"
    ["#{member}_#{name}", Daru::DataFrame.from_csv(file_name).sort(['v'])]
  end.to_h
end.flatten
data = data.first.merge data[1]

data.each do |name, df|
  voltage, current, power = df.map_rows { |row| [row['v'], row['i'], row['i'].abs * row['v'].abs] }.max_by { |v, i, power| power }
  puts "#{name} => #{'%1.2e' % (power * 1000)}(mW), #{current}(I), where V = #{voltage}(V)"
end
nil

sangenya_5cm => 9.27e-01(mW), -0.003(I), where V = 0.309(V)
sangenya_15cm => 2.89e-01(mW), -0.0011(I), where V = 0.263(V)
sangenya_25cm => 2.17e-01(mW), -0.000804(I), where V = 0.27(V)
nagura_5cm => 3.36e-01(mW), -0.0015(I), where V = 0.224(V)
nagura_15cm => 1.49e-01(mW), -0.0007(I), where V = 0.213(V)
nagura_25cm => 8.22e-02(mW), -0.00033(I), where V = 0.249(V)


# n

In [12]:
sangenya_df = Daru::DataFrame.from_csv('data/sangenya/dark_solar_battery.csv').sort(['v'])
sangenya_hash = sangenya_df.to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "sangenya_dark", mode: 'markers+lines')
nagura_df = Daru::DataFrame.from_csv('data/nagura/dark_solar_battery.csv').sort(['v'])
nagura_hash = nagura_df.to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "nagura_dark", mode: 'markers+lines')

sangenya_to_line = sangenya_df.filter_rows { |row| row['v'] > 0.14 }
sangenya_fit = ExpFitting.new(sangenya_to_line['v'].to_a, sangenya_to_line['i'].to_a)
sangenya_fit.fit

nagura_to_line = nagura_df.filter_rows { |row| row['v'] > 0.1 && row['v'] < 0.58 }
nagura_fit = ExpFitting.new(nagura_to_line['v'].to_a, nagura_to_line['i'].to_a)
nagura_fit.fit

Plotly::Plot.new(data: [sangenya_hash, nagura_hash, sangenya_fit.points(0, 1), nagura_fit.points(0, 1)], layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)', type: :log } }).show
nil

#<Plotly::Offline::HTML:0x007fcdabff3a90 @id="5d961fd6-e929-49c2-97ce-f902280ad530", @data=[{:x=>[-0.991, -0.911, -0.874, -0.802, -0.774, -0.734, -0.709, -0.614, -0.538, -0.458, -0.35, -0.142, -0.07529999999999999, -0.0346, -0.0008, 0.105, 0.123, 0.165, 0.19, 0.267, 0.311, 0.339, 0.39, 0.418, 0.44, 0.459, 0.486, 0.505, 0.519, 0.538, 0.554, 0.565, 0.582], :y=>[-0.009000000000000001, -0.0068, -0.005900000000000001, -0.0044, -0.0039, -0.0033, -0.0029, -0.0017, -0.00115, -0.000743, -0.000439, -0.00014, -7.3e-05, -3.3e-05, -8.0e-07, 8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :mode=>"markers+lines", :name=>"sangenya_dark"}, {:x=>[-1.708, -1.626, -1.4, -1.296, -1.209, -1.111, -1.018, -0.929, -0.814, -0.744, -0.62, -0.499, -0.396, -0.308, -0.115, -0.0594, -0.0292, -0.013800000000000002, -0.0009000000000000001, 0.13, 0.272, 0.342, 0.408, 0.417, 0.479, 0.506, 0.54, 0.572, 0.59, 0.617, 0.651, 0.693], :y=>[-0.010400000000000001, -0.0079, -0.0037, -0.0025, -0.0018000000000000002, -0.0011, -0.00071, -0.00044, -0.00022, -0.00017, -6.8e-05, -2.8e-05, -1.4e-05, -7.4e-06, -2.0e-06, -1.0e-06, -5.0e-07, -3.0e-07, 0, 6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :mode=>"markers+lines", :name=>"nagura_dark"}, {:x=>[0, 1], :y=>[3.496088566606789e-05, 0.5204962071065878], :name=>"log y = 9.61e+00 x - 1.03e+01"}, {:x=>[0, 1], :y=>[1.12438847148944e-06, 2.1453111538025276], :name=>"log y = 1.45e+01 x - 1.37e+01"}], @layout={:yaxis=>{:title=>"電流(I)", :type=>:log}, :xaxis=>{:title=>"電圧(V)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>